## 서울시 구별 대기질 데이터 전처리

### 데이터 전처리 (Data Preprocessing)

이 노트북은 서울시 구별 대기질 데이터를 분석에 적합한 형태로 전처리하기 위해 작성되었습니다.

- 데이터 로드 및 기본 정보 확인
- 날짜 형식 변환 및 정렬
- 결측치 처리 및 데이터 보완
- 구별 데이터 통합 및 정제

전처리를 통해 모델링에 필요한 깨끗하고 일관된 형태의 데이터셋을 구축하는 데 목적이 있습니다.

### 1. 데이터 로드 및 기본 설정

In [1]:
#  Jupyter Notebook에서 외부 모듈(.py 파일)을 수정할 경우,
#  매번 커널을 재시작하거나 수동으로 reload하지 않아도
#  자동으로 가장 최신 상태로 import 되도록 설정합니다.

# %load_ext autoreload:
# IPython의 autoreload 확장 기능을 불러옵니다.
# 이 기능은 외부 .py 파일이 변경될 때 자동으로 다시 import 하도록 도와줍니다.

# %autoreload 2:
# 모든 모듈을 cell 실행 시마다 자동으로 reload합니다.
# (즉, 외부 .py 파일을 수정하고 저장만 해도 바로 반영됨)

# 사용 예:
#   - utils.py, visualization.py 등 자주 수정하는 모듈이 있는 경우
#   - 매번 커널 재시작 없이 편리하게 함수 변경 사항을 반영하고 싶을 때

# 단, 모듈의 내부 상태를 유지하고 싶은 경우에는 주의해서 사용해야 합니다.

%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# 프로젝트 루트 경로를 sys.path에 추가
project_root = os.path.abspath("..")  # notebooks 폴더 기준 상위 폴더
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import pandas as pd
from scripts.utils import setup_font

# 한글 폰트 설정
setup_font()

Current OS: Darwin
Setting macOS font: AppleGothic
Current font settings: ['AppleGothic']


### 2. 구별 대기질 데이터 기본 전처리
- 날짜 형식 변환 및 정렬
- 필요한 컬럼 선택
- 기본적인 데이터 정제

In [4]:
from scripts.air_preprocess_utils import preprocess_air_quality_data, save_to_csv, check_missing_data

# 원본 데이터가 저장된 폴더 경로 설정
folder_path = '../data/raw/air_quality/main'
save_dir = '../data/processed/air_quality_raw'

# 처리할 서울시 구별 대기질 데이터 파일 목록
# 각 구의 대기질 데이터가 개별 CSV 파일로 저장되어 있음
files = [
    '강남구.csv', '강동구.csv', '강북구.csv', '강서구.csv', '관악구.csv',
    '광진구.csv', '구로구.csv', '금천구.csv', '노원구.csv', '도봉구.csv',
    '동대문구.csv', '동작구.csv', '마포구.csv', '서대문구.csv', '서초구.csv',
    '성동구.csv', '성북구.csv', '송파구.csv', '양천구.csv', '영등포구.csv',
    '용산구.csv', '은평구.csv', '종로구.csv', '중구.csv', '중랑구.csv'
]

# 결측 데이터 정보를 저장할 딕셔너리 생성
missing_data_dict = {}

# 저장 폴더 생성
os.makedirs(save_dir, exist_ok=True)

# 각 구별 데이터 처리
for file in files:
    # 파일명에서 지역명 추출 (확장자 제거)
    region_name = file.replace('.csv', '')

    # 데이터 파일 경로 생성 및 로드
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)

    # 데이터 전처리
    # - 날짜 형식 변환 및 정렬
    # - 2018년부터 2024년까지의 데이터만 선택
    # - 필요한 컬럼만 유지 (날짜, PM2.5, PM10)
    processed_df = preprocess_air_quality_data(
        df,
        date_col='date',
        start_date='2018-01-01',
        end_date='2024-12-31',
        columns_to_keep=['date', 'pm25', 'pm10']
    )

    # 결측 데이터 확인 및 저장
    missing_info = check_missing_data(processed_df)
    missing_data_dict[region_name] = missing_info

    # 전처리된 데이터 저장
    # '../data/processed/air_quality_raw' 디렉토리에 저장
    save_to_csv(
        processed_df,
        region_name=region_name,
        output_dir='../data/processed/air_quality_raw'
    )

    # 결과 확인
    print(f"{region_name} 전처리 완료!")
    print(f"날짜가 없는 경우: {len(missing_info['날짜 없음'])}개")
    print(f"PM25가 없는 경우: {len(missing_info['PM25 없음'])}개")
    print(f"PM10이 없는 경우: {len(missing_info['PM10 없음'])}개")
    print(f"날짜만 있는 경우: {len(missing_info['날짜만 있음'])}개")

print("모든 구 전처리 완료!!!")

강남구 전처리 완료!
날짜가 없는 경우: 6개
PM25가 없는 경우: 7개
PM10이 없는 경우: 5개
날짜만 있는 경우: 5개
강동구 전처리 완료!
날짜가 없는 경우: 12개
PM25가 없는 경우: 13개
PM10이 없는 경우: 9개
날짜만 있는 경우: 1개
강북구 전처리 완료!
날짜가 없는 경우: 3개
PM25가 없는 경우: 4개
PM10이 없는 경우: 5개
날짜만 있는 경우: 0개
강서구 전처리 완료!
날짜가 없는 경우: 3개
PM25가 없는 경우: 5개
PM10이 없는 경우: 5개
날짜만 있는 경우: 0개
관악구 전처리 완료!
날짜가 없는 경우: 27개
PM25가 없는 경우: 6개
PM10이 없는 경우: 6개
날짜만 있는 경우: 2개
광진구 전처리 완료!
날짜가 없는 경우: 7개
PM25가 없는 경우: 6개
PM10이 없는 경우: 5개
날짜만 있는 경우: 0개
구로구 전처리 완료!
날짜가 없는 경우: 12개
PM25가 없는 경우: 13개
PM10이 없는 경우: 8개
날짜만 있는 경우: 0개
금천구 전처리 완료!
날짜가 없는 경우: 3개
PM25가 없는 경우: 4개
PM10이 없는 경우: 5개
날짜만 있는 경우: 0개
노원구 전처리 완료!
날짜가 없는 경우: 3개
PM25가 없는 경우: 6개
PM10이 없는 경우: 6개
날짜만 있는 경우: 0개
도봉구 전처리 완료!
날짜가 없는 경우: 3개
PM25가 없는 경우: 5개
PM10이 없는 경우: 5개
날짜만 있는 경우: 0개
동대문구 전처리 완료!
날짜가 없는 경우: 58개
PM25가 없는 경우: 10개
PM10이 없는 경우: 6개
날짜만 있는 경우: 0개
동작구 전처리 완료!
날짜가 없는 경우: 3개
PM25가 없는 경우: 7개
PM10이 없는 경우: 6개
날짜만 있는 경우: 0개
마포구 전처리 완료!
날짜가 없는 경우: 35개
PM25가 없는 경우: 8개
PM10이 없는 경우: 8개
날짜만 있는 경우: 0개
서대문구 전처리 완료!
날짜가 없는 경우: 22개
PM25가 없는 경우: 9개
PM10이 없는 경우

### 3. 시간별 데이터 일평균 처리

- 시간별 데이터 로드
- 일별 평균 계산
- 필요한 컬럼 및 구 이름 정제
- 연도별 일평균 데이터 저장

In [5]:
# .xlsx 파일을 읽기 위해 openpyxl 3.1.5 버전 설치 필요
# pip install openpyxl==3.1.5

In [6]:
from scripts.air_preprocess_utils import process_subdata_year_folder
# save_to_csv : 2번 셀에서 중복 정의되어 있음

# 연도별 폴더 경로와 저장 경로 지정
folder_path = '../data/raw/air_quality/sub'
save_dir = '../data/processed/air_sub'

years = [
    2018, 2019, 2020, 2021,
    2022, 2023, 2024
]

# 저장 폴더가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

for year in years:
    year_folder = os.path.join(folder_path, str(year))
    df = process_subdata_year_folder(year_folder)
    
    save_to_csv(
        df,
        region_name=str(year),  # 파일명에 연도 들어가게
        output_dir=save_dir
    )
    
    print(f"{year}년 데이터 저장 완료!")

2018년 데이터 저장 완료!
2019년 데이터 저장 완료!
2020년 데이터 저장 완료!
2021년 데이터 저장 완료!
2022년 데이터 저장 완료!
2023년 데이터 저장 완료!
2024년 데이터 저장 완료!


### 4. 데이터 통합 및 최종 저장
- 기본 전처리 데이터와 일평균 서브 데이터 병합
- 결측치 처리 (3번에서 생성한 서브 데이터 활용)
- 통합 데이터 저장

In [7]:
from scripts.air_preprocess_utils import fill_missing_from_sub
# save_to_csv, check_missing_data : 2번 셀에서 중복 정의되어 있음

# 경로 설정
main_dir = '../data/processed/air_quality_raw'
sub_dir = '../data/processed/air_sub'
save_dir = '../data/processed/air_quality_clean'

"""
    [참고] files 리스트는 2번 셀에서 이미 정의되어 있습니다.
           코드 중복을 피하기 위해 여기서는 재정의하지 않습니다.

    # 구별 파일 목록
    files = [
        '강남구.csv', 강동구.csv', '강북구.csv', '강서구.csv', '관악구.csv',
        '광진구.csv', '구로구.csv', '금천구.csv', '노원구.csv', '도봉구.csv',
        '동대문구.csv', '동작구.csv', '마포구.csv', '서대문구.csv', '서초구.csv',
        '성동구.csv', '성북구.csv', '송파구.csv', '양천구.csv', '영등포구.csv',
        '용산구.csv', '은평구.csv', '종로구.csv', '중구.csv', '중랑구.csv'
    ]
"""

# 결측 데이터 정보를 저장할 딕셔너리 생성
missing_data_dict_2 = {}

# 저장 폴더가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

for file in files:
    region_name = file.replace('.csv', '')
    file_path = os.path.join(main_dir, file)
    df = pd.read_csv(file_path)
    
    # 결측치 채우기
    df_filled = fill_missing_from_sub(
        main_df=df,
        region_name=region_name,
        missing_info=missing_data_dict[region_name],
        sub_dir=sub_dir
    )

    # 결측 데이터 확인 및 저장
    missing_info_2 = check_missing_data(df_filled)
    missing_data_dict_2[region_name] = missing_info_2
    
    # 저장
    save_to_csv(
        df_filled,
        region_name=region_name,
        output_dir=save_dir
    )

    # 결과 확인
    print(f"{region_name} 결측치 보정 및 저장 완료!")
    print(f"날짜가 없는 경우: {len(missing_info_2['날짜 없음'])}개")
    print(f"PM25가 없는 경우: {len(missing_info_2['PM25 없음'])}개")
    print(f"PM10이 없는 경우: {len(missing_info_2['PM10 없음'])}개")
    print(f"날짜만 있는 경우: {len(missing_info_2['날짜만 있음'])}개")

print("모든 구의 결측치 보정이 완료되었습니다!")

강남구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 0개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
강동구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 0개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
강북구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 0개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
강서구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 1개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
관악구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 2개
PM10이 없는 경우: 0개
날짜만 있는 경우: 26개
광진구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 1개
PM10이 없는 경우: 0개
날짜만 있는 경우: 4개
구로구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 1개
PM10이 없는 경우: 0개
날짜만 있는 경우: 9개
금천구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 0개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
노원구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 1개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
도봉구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 1개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
동대문구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 0개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
동작구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25가 없는 경우: 0개
PM10이 없는 경우: 0개
날짜만 있는 경우: 0개
마포구 결측치 보정 및 저장 완료!
날짜가 없는 경우: 0개
PM25

In [8]:
from scripts.air_preprocess_utils import merge_air_quality_files

# 파일 하나로 합치기
merge_air_quality_files('../data/processed/air_quality_clean', '../data/processed/air_quality_merged.csv')

통합 완료! → ../data/processed/air_quality_merged.csv
